# 1. Magyarázatgenerálás "black-box" modellekhez

Készítette: \\
Vetró Mihály \\
Révy Gábor \\
Dr. Hullám Gábor \\

Ebben a példában megvizsgáljuk az összetett modellek kimenetének magyarázhatóságát, egy "kutya-macska" képosztályozási problémán keresztül.

Tekintve, hogy a neurális háló modellek paramétereinek száma bonyolultabb problémák (pl. képfeldolgozó vagy nyelvi modellek) esetén a százezres nagyságrendtől egészen milliárdos nagyságrendig terjed, a modell működésének a paraméterek közvetlen vizsgálatával történő megértése rendkívül nehéz, jellemzően kivitelezhetetlen feladat.
Ebből eredően a modern modellmagyarázó módszerek célja, hogy valamilyen hatásvizsgálat segítségével megállapítsák, hogy a modell kimenetének meghatározásában mely bemeneti paraméterek játszanak jelentős szerepet, így betekintést engedve azon "szempontrendszerbe", amely alapján a predikció előállt.

A leggyakoribb magyarázatgenerálásra használt módszerek alapvetően két fő csoportba sorolhatók: (1) a modellfüggetlen módszerek (nevükből is sejthető módon) pusztán a modell bemenetére, és az arra adott kimenetre ("válaszra") koncentrálnak, míg (2) a modellspecifikus módszerek a modell struktúráját és paramétereit is figyelembe veszik a hatásvizsgálat során.

A gyakorlati rész megkezdése előtt fontos megjegyezni, hogy habár a jelen példában képosztályozó modellek magyarázásával foglalkozunk, más modelltípusok is magyarázhatóak hasonló elven alapuló módszerek segítségével.


Első, előkészítő lépésként állítsuk alaphelyzetbe a futtatókörnyezetet, majd végezzük el a szükséges könyvtárak telepítését (`lime` és `alibi`) és importálását az alábbi kódblokkok futtatásával:

In [ ]:
# install required libraries:
!pip install lime
!pip install alibi[tensorflow]

Ezután indítsuk újra a futtatókörnyezetet, majd folytassuk az alábbi kódblokk futtatásával!

In [ ]:
%%capture

# delete previous variables:
%reset -f

# import required packages:
import os
import cv2
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from alibi.explainers import IntegratedGradients

Ezt követően töltsük be az adatot:

In [ ]:
IMAGE_RESOLUTION = 256

(train_dataset, val_dataset, test_dataset), dataset_info = \
            tfds.load('cats_vs_dogs',
                      split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                      shuffle_files=False, with_info=True, as_supervised=True)

classes = dataset_info.features['label'].names  # label name of the classes
n_classes = len(classes)  # number of classes

test_images, test_labels = [], []

for img, lab in test_dataset.take(200):
    img_resized = tf.image.resize_with_pad(img, IMAGE_RESOLUTION, IMAGE_RESOLUTION) / 255.
    test_images.append(img_resized.numpy())
    test_labels.append(lab.numpy().flatten())

test_images = np.array(test_images)
test_labels = np.array(test_labels)

Ehhez a demonstrációhoz az előző feladatnál használt architektúra egy módosított, és előtanított változatát fogjuk használni, az eredmények egységessége érdekében.

Töltsük be ezt a modellt az alábbi kódblokk lefuttatásával:

In [ ]:
from urllib import request

model_address = "https://share.mit.bme.hu/index.php/s/mYMKQb2SXGooa5L/download/dvc_classification_model.h5"
request.urlretrieve(model_address, "dvc_model_revised.h5")

dvc_classification_model = tf.keras.models.load_model("dvc_model_revised.h5")

dvc_classification_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy'])

## Modellfüggetlen magyarázat - LIME

Elsőként vizsgáljuk meg a Local Interpretable Model-Agnostic Explanations (röviden LIME) magyarázó módszert.

Ezen módszer működését röviden az alábbi lépéssor foglalja össze:

1. Mintavételezzük a bemeneti teret a magyarázni kívánt bemenet körül (pl. a bemenethez társított véletlenszerű additív zaj segítségével)
2. Rögzítsük a modell predikcióit a mintavételezett pontokra.
3. Illesszünk egy egyszerű (jellemzően lineáris) modellt az így kapott bemenet-kimenet párokra úgy, hogy a modell elkészítése során a mintákat az eredeti (magyarázni kívánt) bemenettől való távolság inverzével súlyozzuk.
4. Az így kapott egyszerű modell magyarázható, és a vizsgált bemenet környezetében feltehetően jól imitálja az eredeti modell viselkedését, tehát illeszkedik a döntési felületre. Az egyszerű modell paramétereiből megállapíthatjuk az egyes bemenetek szerepét az osztályozó döntésének kialakításában.

A fent prezentált lépéssor első három elemét az alábbi ábra foglalja össze:

![LIME image](https://raw.githubusercontent.com/marcotcr/lime/master/doc/images/lime.png)

Ahol a magyarázni kívánt minta piros kereszttel, a bemeneti térből vett minták keresztekkel és körökkel, az eredeti modell döntési felületét a háttér színezésével, a mintákra illesztett magyarázható (lineáris) modell döntési felületét pedig szaggatott vonallal jelölték.

A LIME módszer részletes leírása a [kapcsolódó publikációban](https://arxiv.org/pdf/1602.04938v3.pdf) fellelhető.

> Összefoglalva tehát a LIME módszer pusztán a bemenetek, és a modell azokra adott válasza alapján definiál egy magyarázható modellt, amely közelíti a magyarázandó modell viselkedését a bemenet környezetében. Mivel a LIME nem követel meg semmilyen feltételezést a magyarázandó modell belső működésével kapcsolatban, így bármilyen prediktív modell kimenetének magyarázására felhasználható, tehát modellfüggetlen módszer.

Esetünkben a LIME módszer egy kifejezetten képi bemenetekhez megalkotott változatát fogjuk használni, ami a fent bevezetett alapokon felül a bemeneteket textúra vagy más előre megadott függvény alapján összefüggő részekre, úgynevezett szuperpixelekre osztja, majd az egyes szuperpixelek attribúcióit állapítja meg. Ez az előzetes lépés kritikus, ugyanis ha a kép akár több százezer pixelét külön változóknak vennénk, akkor rendkívül sok mintára lenne szükség a bemeneti tér lokális lefedéséhez.

Alapértelmezésben a LIME a szuperpixelekre való szegmentáláshoz a Quickshift algoritmust használja, amely módszer működését a [kapcsolódó publikáció](https://link.springer.com/chapter/10.1007/978-3-540-88693-8_52) részletesen kifejti.

Az alábbi kódblokkban definiáljuk a LIME Explainert, és előkészítjük a magyarázatgenerálást véletlenszerűen kiválasztott képekhez:

In [ ]:
n_correct_imgs_to_explain = 4
n_incorrect_imgs_to_explain = 4

test_predictions = np.squeeze(dvc_classification_model.predict(test_images, verbose=0))

correct_preds_mask = test_predictions.flatten().round().astype(int) == test_labels.flatten()
imgs_correct, labels_correct = test_images[correct_preds_mask], test_labels[correct_preds_mask]
imgs_incorrect, labels_incorrect = test_images[np.logical_not(correct_preds_mask)], test_labels[np.logical_not(correct_preds_mask)]


selected_indices = (np.random.choice(len(imgs_correct), n_correct_imgs_to_explain, replace=False),
                    np.random.choice(len(imgs_incorrect), n_incorrect_imgs_to_explain, replace=False))
selected_images, selected_labels = (np.concatenate([imgs_correct[selected_indices[0]], imgs_incorrect[selected_indices[1]]]),
                                    np.concatenate([labels_correct[selected_indices[0]], labels_incorrect[selected_indices[1]]]))

explainer = lime_image.LimeImageExplainer()

test_predictions_lime = np.concatenate([test_predictions[correct_preds_mask][selected_indices[0]],
                                        test_predictions[~correct_preds_mask][selected_indices[1]]])

classifier_fn = lambda x: dvc_classification_model.predict(x, verbose=0)

Alább az eredmények megjelenítéséhez és grid szegmentációhoz definiálunk segédfüggvényeket.

In [ ]:
from skimage.segmentation import quickshift, mark_boundaries

random_seed = 42

def plot_explanation(explanations, predictions, labels, num_superpixels_per_class=None, min_weight=None):
  n_images = len(test_predictions_lime)
  n_images_per_row = 4
  n_rows = (n_images - 1) // n_images_per_row + 1
  fig, axes = plt.subplots(n_rows, n_images_per_row, figsize=(12, 16))
  for prediction, explanation, label, ax in zip(predictions, explanations, labels, axes.flatten()):
      temp, mask_dog = explanation.get_image_and_mask(0, positive_only=True,
                                                      num_features=num_superpixels_per_class, hide_rest=False,
                                                      min_weight=min_weight)
      temp, mask_cat = explanation.get_image_and_mask(0, positive_only=False, negative_only=True,
                                                      num_features=num_superpixels_per_class, hide_rest=False,
                                                      min_weight=min_weight)

      mask_transparent = np.full(temp.shape[:2], np.nan)
      mask_transparent[mask_dog.astype(bool)] = 1
      mask_transparent[mask_cat.astype(bool)] = 0

      ax.imshow(temp)
      # Yellow: dog --- Blue: cat
      ax.imshow(mask_transparent, cmap="plasma", alpha=0.5)
      ax.axis("off")
      ax.set_title(f"{'cat' if label == 0 else 'dog'}\ncat:  {(1. - prediction):.2f}\ndog: {prediction:.2f}", color="green" if np.round(prediction) == label else "red")

  plt.tight_layout()
  plt.show();

def create_grid(image, grid_size=(10, 10)):
  image_h, image_w = image.shape[:2]
  n_grid_y, n_grid_x = grid_size
  grid_size_y, grid_size_x = image_h // n_grid_y, image_w // n_grid_x
  pixel_indices = np.mgrid[0:image_h, 0:image_w]
  grid_indices = (pixel_indices[0] // grid_size_y) * n_grid_x + pixel_indices[1] // grid_size_x

  return grid_indices

Végül generáljuk le a magyarázatokat különböző szegmentáló függvények mellett és ábrázoljuk a legjelentősebb szuperpixeleket mindkét osztály mellett:

In [ ]:
segmentation_fn = lambda img: create_grid(img, (20, 20))
plt.imshow(mark_boundaries(selected_images[0], segmentation_fn(selected_images[0])))
plt.show()

test_explanations_grid_small = []
for img in selected_images:
  test_explanations_grid_small.append(explainer.explain_instance(img, classifier_fn=classifier_fn, top_labels=1, num_samples=1000, segmentation_fn=segmentation_fn))

In [ ]:
plot_explanation(test_explanations_grid_small, test_predictions_lime, selected_labels, min_weight=0.05)

In [ ]:
segmentation_fn = lambda img: create_grid(img, (10, 10))
plt.imshow(mark_boundaries(selected_images[0], segmentation_fn(selected_images[0])))
plt.show()

test_explanations_grid_medium = []
for img in selected_images:
  test_explanations_grid_medium.append(explainer.explain_instance(img, classifier_fn=classifier_fn, top_labels=1, num_samples=500, segmentation_fn=segmentation_fn))

In [ ]:
plot_explanation(test_explanations_grid_medium, test_predictions_lime, selected_labels, min_weight=0.05)

In [ ]:
segmentation_fn = lambda img: create_grid(img, (5, 5))
plt.imshow(mark_boundaries(selected_images[0], segmentation_fn(selected_images[0])))
plt.show()

test_explanations_grid_large = []
for img in selected_images:
  test_explanations_grid_large.append(explainer.explain_instance(img, classifier_fn=classifier_fn, top_labels=1, num_samples=500, segmentation_fn=segmentation_fn))

In [ ]:
plot_explanation(test_explanations_grid_large, test_predictions_lime, selected_labels, min_weight=0.1)

In [ ]:
segmentation_fn = lambda img: quickshift(img, kernel_size=3, max_dist=200, ratio=0.2, rng=random_seed)
plt.imshow(mark_boundaries(selected_images[0], segmentation_fn(selected_images[0])))
plt.show()

test_explanations_quickshift_small = []
for img in selected_images:
  test_explanations_quickshift_small.append(explainer.explain_instance(img, classifier_fn=classifier_fn, top_labels=1, num_samples=1000, segmentation_fn=segmentation_fn))

In [ ]:
plot_explanation(test_explanations_quickshift_small, test_predictions_lime, selected_labels, min_weight=0.05)

In [ ]:
segmentation_fn = lambda img: quickshift(img, kernel_size=6, max_dist=200, ratio=0.2, rng=random_seed)
plt.imshow(mark_boundaries(selected_images[0], segmentation_fn(selected_images[0])))
plt.show()

from time import time
start = time()

test_explanations_quickshift_medium = []
for img in selected_images:
  test_explanations_quickshift_medium.append(explainer.explain_instance(img, classifier_fn=classifier_fn, top_labels=1, num_samples=1000, segmentation_fn=segmentation_fn))

print(f'elapsed time: {time() - start}')

In [ ]:
plot_explanation(test_explanations_quickshift_medium, test_predictions_lime, selected_labels, min_weight=0.1)

In [ ]:
segmentation_fn = lambda img: quickshift(img, kernel_size=10, max_dist=200, ratio=0.2, rng=random_seed)
plt.imshow(mark_boundaries(selected_images[0], segmentation_fn(selected_images[0])))
plt.show()

test_explanations_quickshift_large = []
for img in selected_images:
  test_explanations_quickshift_large.append(explainer.explain_instance(img, classifier_fn=classifier_fn, top_labels=1, num_samples=1000, segmentation_fn=segmentation_fn))

In [ ]:
plot_explanation(test_explanations_quickshift_large, test_predictions_lime, selected_labels, min_weight=0.15)

A kódblokkok futása ~18 percet vesz igénybe.

Amíg a magyarázatok elkészülnek, válaszoljon az alábbi kérdésre:

**Előzetes feltételezése szerint egy adott kép mely része releváns, illetve mely részei irrelevánsak az osztályozás szempontjából?**
\
*Tipp: Referenciaként tekintsen vissza az előző feladatban kirajzolt mintaképekre.*
```
TODO: válasz
```

A fenti eredményeken a modell által leginkább figyelembe vett szuperpixelek vannak kiemelve, sárga vagy kék színezéssel. A a sárga szín azon szuperpixeleket jelöli, amelyek a modell döntését az 1-es (kutya), a kék szín pedig azokat amik a modell döntését a 0-s (macska) osztály felé tolja el. Emellett minden kép fölött szerepel a modell által adott predikció szöveges formában, amely szöveg színezése azt jelöli, hogy a modell predikciója helyes-e vagy sem (zöld: helyes predikció, piros: hibás predikció)

Figyelem: Mindkét osztály "mellett" az N darab (alapértelmezetten 5) legjelentősebb szuperpixel lett bejelölve. Ebből adódóan mindkét osztály szuperpixelei jelen vannak minden képen.

A fenti információk és az eredmények függvényében válaszolja meg az alábbi kérdéseket:

**Lát-e valamilyen összefüggést a LIME módszer által adott attribúciók ("magyarázatok") és a modell döntése, illetve a modell döntésének helyessége között? Ha igen, akkor mi lehet ennek az oka?**
```
TODO: válasz
```

**Megfigyelhető-e valamilyen összefüggés a modell bizonytalansága (tehát hogy a predikció mennyire van közel a 0.5-ös "teljesen bizonytalan" értékhez) és a LIME által meghatározott attribúciók között?**
```
TODO: válasz
```

**Hogyan változnak az attribúciók a kiemelt szuperpixelek számának függvényében? Az esetleges változás milyen hatással van az előző kérdésekben fölvetett szempontokra?**
\
*Tipp: a szuperpixelek osztályonkénti számát a fenti kódblokk elején található `num_superpixels_per_class` változóval, lehet beállítani, ami után a kódblokkot értelemszerűen újra kell futtatni. A szuperpixelek száma osztályonként alapértelmezetten 5.*
```
TODO: válasz
```

## Modellspecifikus magyarázat - Integrált Gradiens

Az Integrált Gradiens módszer kihasználja a neurális hálók azon tulajdonságát, hogy - ideális esetben - az általuk megvalósított leképezés teljes egészében deriválható (ez ugyanaz a tulajdonság, amelyet kihasználva a neurális hálók tanítása is zajlik).

A módszer nagyvonalakban az alábbi lépésekből áll:
1. Határozzunk meg egy referenciapontot a bemeneti térben (képosztályozásnál ez jellemzően egy teljesen fekete kép), majd interpoláljunk egy egyenes szakaszt ezen referenciapont és a magyarázni kívánt bemenet között.
2. Az interpolált egyenes mentén adott lépésközzel haladva állítsunk elő példabemeneteket. - Ezen egyenesen haladva minden előállított kép egy átmenet lesz a referencia kép és a magyarázni kívánt kép között.
3. Minden előállított bemenetre számítsuk ki a kimenet deriváltját a bemenet változóinak (pixeleinek) függvényében.
4. Az így előállított gradiensek értékének becsüljük meg az integrálját az interpolált szakaszon. Az így kapott integrál értéke lesz az adott változóhoz társított attribúció (tehát hogy a kimenetet milyen irányban, és mekkora mértékben befolyásolja).

A módszer részletes leírása a kapcsolódó cikkben olvasható: [Axiomatic Attribution for Deep Networks](https://arxiv.org/pdf/1703.01365.pdf)

> Fontos megjegyezni, hogy ezen módszer eredménye bizonyos esetekben nagy mértékben függhet a felhasznált referenciaponttól. A "baseline" (vagyis referencia-bemenet) megválasztásának hatásáról az alábbi oldal ad bővebb leírást: [Visualizing the Impact of Feature Attribution Baselines](https://distill.pub/2020/attribution-baselines/)

> További fontos, előzetes megjegyzés, hogy az Integrált Gradiens módszer pixelek szintjén állapítja meg a bemenet attribúcióit, szemben a LIME módszerrel, amely szuperpixelek szintjén működik a mintavételezés sajátosságai miatt.

Az alábbi kódblokk segítségével inicializáljuk az Integrált Gradiens módszert megvalósító osztályt, majd számítsuk ki az előző példában kiválasztott bemenetek attribúcióit:

In [ ]:
ig = IntegratedGradients(dvc_classification_model,
                         layer=None,
                         method="gausslegendre",
                         n_steps=50,
                         internal_batch_size=100)

test_explanations_intgrad = []

for img, prediction in zip(selected_images, np.array(test_predictions).flatten()):
  test_explanations_intgrad.append(ig.explain(img[None, :, :, :], target=0))

Majd rajzoljuk ki a kapott attribúció-értékeket:

In [ ]:
inclusion_percentile = 5.
cutoff_rate = 0.25

fig, axes = plt.subplots(4, 4, figsize=(12, 16))

for prediction, explanation, label, image, ax in zip(np.array(test_predictions).flatten(), test_explanations_intgrad, selected_labels.flatten(), selected_images, axes.flatten()):
    avg_decolored_mask = explanation.attributions[0][0].mean(axis=-1)
    mask_dog = avg_decolored_mask < np.percentile(avg_decolored_mask, inclusion_percentile)
    mask_cat = avg_decolored_mask > np.percentile(avg_decolored_mask, 100. - inclusion_percentile)

    mask = mask_dog + mask_cat
    mask = np.tile(mask[:, :, None], 3).astype(np.uint8) * 255
    mask_blurred = cv2.GaussianBlur(mask, (13,13), 11)
    mask_blurred_capped = mask_blurred.mean(axis=-1)
    mask_blurred_capped[mask_blurred_capped < np.round(cutoff_rate * 255.)] = np.nan

    ax.imshow(image)
    ax.imshow(mask_blurred_capped, cmap="Reds", vmin=0, vmax=255, alpha=0.5)

    ax.axis("off")
    ax.set_title(f"{'cat' if label == 0 else 'dog'}\ncat:  {(1. - prediction):.2f}\ndog: {prediction:.2f}", color="green" if np.round(prediction) == label else "red")

plt.tight_layout()
plt.show();

Mivel az Integrált Gradiens módszer kivétel nélkül minden pixelhez társít egy attribúció-értéket, így az átlátható megjelenítés érdekében a fenti megjelenítés csak a szélsőértékeken lévő pixel-értékek által lefedett területeket jeleníti meg, illetve a megjelenített pixelek láthatóságát egy Gauss-elmosás tovább növeli.
\
A kódblokk elején található `inclusion_percentile` változó meghatározza, hogy mely alsó- és felső percentilis fölé (illetve alá) eső attribúció-értékek kerülnek megjelenítésre. Alapértelmezetten az értéke 5, ami azt jelenti, hogy az 5. percentilis alatti, és a (100 - 5 =) 95. percentilis fölötti attribúciókat jelenítjük meg. Ennek a változónak az értéke 0-50 között mozoghat.
\
A másik, megjelenítést szabályozó változó a `cutoff_rate`, amely azt határozza meg, hogy mekkora az a minimum relatív attribúciós érték, amely alatt az attribúciós maszk teljesen áttetszővé válik (tehát csak az eredeti pixel-érték jelenik meg a képen). Az értéke alapértelmezetten 0.25, és 0-1 között mozoghat.

> Habár az Integrált Gradiens módszer (a LIME-hoz hasonlóan) képes elkülöníteni az egyes osztályokhoz tartozó attribúciókat, ezek pixelről pixelre erősen változnak, így tehát a két osztályhoz attributált területek szinte minden esetben átlapolódnak. Ebből adódóan a fenti példán osztályoktól függetlenül, pusztán az attribúció 0-tól való eltérése alapján történik a megjelenítés.

A fent látottakat és a módszerek sajátosságait figyelembe véve válaszolja meg az alábbi kérdéseket:

**Lát összefüggést az osztályozás helyessége és az attribúciók között? Miben mások az attribúciós maszkok a LIME-nál látottakhoz képest?**
```
TODO: válasz
```

**Mi az eltérés az itt tapasztalt attribúciós maszkok és a LIME-nál látott attribúciók között a kép sajátosságait illetően? Hogyan függ ez a viszony a kódblokk elején beállított, megjelenítéshez használatos paraméterektől?**
\
*Tipp: Figyeljük meg, hogy az egyes módszerek által előállított maszkok mennyire illeszkednek a képen található természetes "határokra" (élekre).*
```
TODO: válasz
```